https://www.loc.gov/free-to-use/classic-childrens-books/

https://concretecomputing.com/thoughts/list-of-public-domain-free-books-for-kids-by-grade-level/

In [12]:
from ebooklib import epub
import os
import ebooklib
import os
import nltk
##import the required libraries and APIs
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
ignore_list = ["<class 'ebooklib.epub.EpubImage'>","<class 'ebooklib.epub.EpubItem'>"]

In [14]:
all_books = []
for filename in os.listdir('books/'):
    if filename.endswith('pg11703.epub'):
        book = epub.read_epub('books/' + filename)
        book_text = ''
        for doc in book.get_items():
            if not str(type(doc)) in ignore_list:
                doc_content = doc.content.decode()
                book_text += doc_content
        book_text = book_text.split('\n')
        for line in book_text:
            if line.startswith('<p>'):
                line = line.replace('<p>','').replace('</p>','')
                line = line.split(".")
                for sent in line:
                    if sent.startswith(" "):
                        sent = sent[1:]
                    if sent.endswith(" "):
                        sent = sent[:-1]
                    if len(sent) > 0:
                        all_books.append(sent.lower())

In [15]:
##set up tokenizer
tokenizer = Tokenizer()

In [16]:
tokenizer.fit_on_texts(all_books)

##calculate vocabulary size - be mindful of the <oov> token
vocab_size = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(vocab_size)

{'the': 1, 'and': 2, 'to': 3, 'of': 4, 'a': 5, 'i': 6, 'we': 7, 'in': 8, 'was': 9, 'it': 10, 'with': 11, 'our': 12, 'had': 13, 'my': 14, 'which': 15, 'he': 16, 'for': 17, 'his': 18, 'that': 19, '”': 20, 'as': 21, 'on': 22, 'but': 23, 'them': 24, 'by': 25, 'not': 26, 'were': 27, 'they': 28, 'us': 29, 'at': 30, 'this': 31, 'be': 32, 'him': 33, 'from': 34, 'some': 35, 'all': 36, 'me': 37, 'have': 38, 'her': 39, 'their': 40, 'you': 41, 'fritz': 42, 'said': 43, 'could': 44, 'title': 45, 'id': 46, 'class': 47, 'x': 48, 'ebookmaker': 49, 'pageno': 50, 'pg': 51, 'is': 52, 'would': 53, 'then': 54, 'she': 55, 'jack': 56, 'ernest': 57, 'little': 58, 'when': 59, 'who': 60, 'one': 61, 'so': 62, 'out': 63, 'might': 64, 'very': 65, 'made': 66, 'an': 67, 'or': 68, 'wife': 69, 'these': 70, 'if': 71, 'will': 72, 'should': 73, 'soon': 74, 'into': 75, 'time': 76, 'no': 77, 'now': 78, 'more': 79, 'are': 80, 'two': 81, 'after': 82, 'mother': 83, 'found': 84, 'up': 85, 'great': 86, 'br': 87, 'saw': 88, 'ther

In [17]:
##create sequences of 
input_sequences = []
for line in all_books:
    tokens = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(tokens)):
        n_gram_sequence = tokens[:i+1]
        input_sequences.append(n_gram_sequence)

In [18]:
##pad sequences
max_seq_len = max([len(i) for i in input_sequences])
input_seq_array = np.array(pad_sequences(input_sequences,
                                         maxlen=max_seq_len,
                                         padding='pre')
                        )

In [19]:
##creating features(X) and label(y)
X = input_seq_array[:, :-1]
labels = input_seq_array[:, -1]

##one-hot encode the labels to get y - since it is actually just a classification problem
y = tf.keras.utils.to_categorical(labels, num_classes=vocab_size)

In [20]:
model = tf.keras.Sequential([
                tf.keras.layers.Embedding(vocab_size, 120, input_length=max_seq_len-1),
                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(120)),
                tf.keras.layers.Dense(vocab_size, activation='softmax')
])

##define the learning rate - step size for optimizer
adam = tf.keras.optimizers.Adam(lr=0.01)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
3520/3520 [==============================] - 1093s 309ms/step - loss: 6.0691 - accuracy: 0.1323
Epoch 2/20
3520/3520 [==============================] - 1091s 310ms/step - loss: 5.4259 - accuracy: 0.1636
Epoch 3/20
3520/3520 [==============================] - 931s 264ms/step - loss: 5.1012 - accuracy: 0.1732
Epoch 4/20
3520/3520 [==============================] - 866s 246ms/step - loss: 4.8946 - accuracy: 0.1798
Epoch 5/20
3520/3520 [==============================] - 850s 242ms/step - loss: 4.7840 - accuracy: 0.1831
Epoch 6/20
3520/3520 [==============================] - 851s 242ms/step - loss: 4.7055 - accuracy: 0.1860
Epoch 7/20
3520/3520 [==============================] - 852s 242ms/step - loss: 4.6493 - accuracy: 0.1891
Epoch 8/20
3520/3520 [==============================] - 851s 242ms/step - loss: 4.6057 - accuracy: 0.1923
Epoch 9/20
3520/3520 [==============================] - 854s 242ms/step - loss: 4.5742 - accuracy: 0.1933
Epoch 10/20
3520/3520 [=====================

KeyboardInterrupt: 